# Table of Contents
* [Learning Objectives:](#Learning-Objectives:)
* [Sales data](#Sales-data)
* [Purchasing patterns](#Purchasing-patterns)
	* [Pivot table](#Pivot-table)
	* [Nested Pivot tables](#Nested-Pivot-tables)
		* [Compare to IndexSlice](#Compare-to-IndexSlice)


# Learning Objectives:

After completion of this module, learners should be able to:

* Read data from multiple files into a single data frame
* Construct pivot tables for complex grouping and aggregation tasks
* Use cross section (`.xs`) to make selections from a hierarchical DataFrame

# Sales data

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

# Purchasing patterns

This data comes from [Practical Business Python](http://pbpython.com/).

Let's start by loading the sales data from 2014.

The `sample-sales.csv` file contains 1000 purchase entries for a fictitious company. This company sells shirts, shoes and belts.

In [ ]:
items_sold = pd.read_csv('data/pbpython/sample-sales.csv')
items_sold['date'] = pd.to_datetime(items_sold['date'])

In [ ]:
items_sold.info()

In [ ]:
items_sold.head()

The `include=` keyword argument to `.describe()` allows statistical operations to be performed on no-numeric columns. In this data set we see that there are 718 unique account names and 3 unique categories.

In [ ]:
items_sold.describe(include=['object'])

In [ ]:
items_sold['category'].unique()

## Pivot table

In a previous exercise we performed a `groupby` operation followed by an aggregation method and a call to `.unique()` to determine how many unique Account Names bought either belts, shirts or shoes.

Using the `items_sold` DataFrame I want the following information in a single table

1. Group the data by the three unique *categories*
1. For each *category* group the *quantity* by *Account Name* and add the results

Doing it only with `groubpy` can be a bit hard as two calls are required and the resulting series objects need to be recombined into a DataFrame with correct column names. There may be more than one way of doing this operation with `groupby`.

In [ ]:
# the hard way
grp = items_sold.groupby('category')

dfs = {i:grp.get_group(i).groupby('Account Name')['quantity'].sum()
       for i in grp.groups}

pd.DataFrame(dfs).head()

A *pivot table* can be used to create a single DataFrame that performs multiple `groupby`, aggregation and re-indexing operations.

The input arguments to `pivot_table` are
* `values`: the column over which to aggregate
* `index`: the column to be used as the index at the end
* `columns`: the column used to group the `values` by
* `aggfunc`: the aggregation method to perform

In [ ]:
categories = pd.pivot_table(items_sold,
               values='quantity', 
               index='Account Name',
               columns='category',
               aggfunc=np.sum)
categories.head()

With a single DataFrame I can iterate over the columns `Belt`, `Shirt` and `Shoes` to determine how many customers bought each of the three items.

Notice the use of `.notnull()`. This creates a bool array of False where the value is `NaN` and True otherwise.

In [ ]:
# loop over columns
for item in categories:
    idx = categories[item].notnull()
    amount = len(categories.loc[idx,item])
    print("%d customers bought %s" % (amount,item))

Because I have all of the data contained in columns I can ask further questions by creating fancy indexing arrays.

For example, how many customers bought `Shoes` **and** `Belts`?

In [ ]:
idx = categories['Shirt'].notnull() & categories['Shoes'].notnull()
shoes_and_belts = categories[idx].shape[0]
print("%d customers bought shoes and belts" % shoes_and_belts)

How many customers bought `Shirts` **but not** `Belts`. The `.isnull()` method is the logical reverse of `.notnull()`

In [ ]:
idx = categories['Shirt'].notnull() & categories['Belt'].isnull()
shoes_not_belts = categories[idx].shape[0]
print("%d customers bought shoes and not belts" % shoes_not_belts)

Finally, our favorite customers are those who bought Shirts **and** Belts **and** Shoes are these are the rows that have no `NaN` values so `.dropna()` is the most convenient method.

In [ ]:
our_favorites = categories.dropna(how='any') # how='any' is default
our_favorites

## Nested Pivot tables

The four keyword arguments used above, `index`, `values`, `columns` and `aggfunc` can each take a list of values to add more columns or construct hierarchical indexes and columns.

For example, the `salesfunnel.xlsx` data set represents sales activities for several `Products` that are sold by the company along with the account name, the sales representative and the sales manager. A hierarchical approach to this data set is 

* A `Manager` may oversee multiple `Reps`
  * A `Rep` will sell each of the `Products`
    * A number of different customers are managed by each sales `Rep`

In [ ]:
df = pd.read_excel("data/pbpython/salesfunnel.xlsx")
df.head()

We can use a pivot table to build the hierarchical indexing along with performing aggregations on the numeric quantities `Quantity` and `Price`. This gives us a global view of the data set that we can use to determine employee and manager performance.

I want to know the total and average quantities of each of the three products sold per sales representative and group them by sales manager.

In [ ]:
table = pd.pivot_table(df,
                       index=["Manager","Rep","Product"],
                       values=["Price","Quantity"],
                       aggfunc=[np.sum,np.mean])
table

To help with performing selections on MultiIndexes the cross section (`.xs()`) method allows easy selection of a level and a value to select from the whole DataFrame.

For instance, we can see all sales data by `Reps` managed by *Fred Anderson*. The cross section method will always **drop** the level that is being selected. It can be disabled with the `drop_level=` keyword argument.

In [ ]:
table.xs('Fred Anderson', level='Manager', drop_level=True)

Values from multiple levels can be provided in a container. It is also possible to chain cross section calls.

In [ ]:
table.xs(['Debra Henley','Daniel Hilton']).xs('Quantity', level=1, axis='columns')

### Compare to IndexSlice

The cross section method is not only less cumbersome, but is generally a bit faster than using the standard MultiIndex slicing.

In [ ]:
%timeit table.xs(['Debra Henley','Daniel Hilton']).xs('Quantity', level=1, axis='columns')

In [ ]:
%%timeit
idx = pd.IndexSlice
table.loc[idx['Debra Henley', 'Daniel Hilton' , :], idx[:, 'Quantity']]